In [1]:
# Import necessary libraries
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
# Define dataset directory
dataset_dir = r"C:\Users\rusha\OneDrive\Desktop\Dental\Dental_Data_Set"
# Image parameters
img_width, img_height = 512, 512  # Increased resolution
batch_size = 8
epochs = 15  # Reduce and use fine-tuning later
learning_rate = 0.0001  # Reduced for stable training
# Load preprocessed train and validation data
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.2).flow_from_directory(
    dataset_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Training data
)

In [2]:
# Define dataset directory
dataset_dir = r"C:\Users\rusha\OneDrive\Desktop\Dental\Dental_Data_Set"

In [3]:
# Image parameters
img_width, img_height = 512, 512  # Increased resolution
batch_size = 8
epochs = 15  # Reduce and use fine-tuning later
learning_rate = 0.0001  # Reduced for stable training

In [4]:
# Load preprocessed train and validation data
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.2).flow_from_directory(
    dataset_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Training data
)

Found 2710 images belonging to 7 classes.


In [5]:
val_generator = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.2).flow_from_directory(
    dataset_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Validation data
)

Found 674 images belonging to 7 classes.


In [6]:
# Define the CNN model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_width, img_height, 3)),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Flatten(),  # Flatten the feature maps
    Dense(256, activation='relu'),
    Dropout(0.5),  # Regularization
    Dense(7, activation='softmax')  # Output layer with 7 classes
])

C:\Users\rusha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
# Compile the model
model.compile(
    optimizer=Adam(learning_rate=learning_rate),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [8]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs
)

C:\Users\rusha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
339/339 ━━━━━━━━━━━━━━━━━━━━ 1632s 5s/step - accuracy: 0.1962 - loss: 30.8768 - val_accuracy: 0.3071 - val_loss: 9.2117
Epoch 2/15
339/339 ━━━━━━━━━━━━━━━━━━━━ 1526s 4s/step - accuracy: 0.4530 - loss: 7.8002 - val_accuracy: 0.2923 - val_loss: 14.5572
Epoch 3/15
339/339 ━━━━━━━━━━━━━━━━━━━━ 1510s 4s/step - accuracy: 0.5080 - loss: 4.0980 - val_accuracy: 0.2774 - val_loss: 10.2573
Epoch 4/15
339/339 ━━━━━━━━━━━━━━━━━━━━ 1485s 4s/step - accuracy: 0.5547 - loss: 2.6012 - val_accuracy: 0.2552 - val_loss: 8.8225
Epoch 5/15
339/339 ━━━━━━━━━━━━━━━━━━━━ 1342s 4s/step - accuracy: 0.5699 - loss: 2.3678 - val_accuracy: 0.2418 - val_loss: 8.5227
Epoch 6/15
339/339 ━━━━━━━━━━━━━━━━━━━━ 1338s 4s/step - accuracy: 0.5396 - loss: 2.5427 - val_accuracy: 0.2478 - val_loss: 13.6711
Epoch 7/15
339/339 ━━━━━━━━━━━━━━━━━━━━ 1335s 4s/step - accuracy: 0.6085 - loss: 1.7482 - val_accuracy: 0.2493 - val_loss: 7.8121
Epoch 8/15
339/339 ━━━━━━━━━━━━━━━━━━━━ 1339s 4s/step - accuracy: 0.6156 - loss: 1.721

In [9]:
# Save the model
model.save("dentalFinalModel.h5")
print("Model training completed and saved.")



Model training completed and saved.


In [10]:
# Evaluate the model
test_generator = tf.keras.preprocessing.image.ImageDataGenerator().flow_from_directory(
    dataset_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 3384 images belonging to 7 classes.


In [11]:
# Predictions and evaluation
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = test_generator.classes

423/423 ━━━━━━━━━━━━━━━━━━━━ 237s 555ms/step


In [12]:
# Display evaluation metrics
print("Classification Report:\n", classification_report(y_true_classes, y_pred_classes))
print("Confusion Matrix:\n", confusion_matrix(y_true_classes, y_pred_classes))

Classification Report:
               precision    recall  f1-score   support

           0       0.71      0.60      0.65       491
           1       0.72      0.70      0.71       486
           2       0.74      0.62      0.68       473
           3       0.70      0.73      0.71       490
           4       0.43      0.75      0.54       488
           5       0.74      0.51      0.61       472
           6       0.80      0.69      0.74       484

    accuracy                           0.66      3384
   macro avg       0.69      0.66      0.66      3384
weighted avg       0.69      0.66      0.66      3384

Confusion Matrix:
 [[294  24  19  23  88  23  20]
 [  4 341  10  37  75  10   9]
 [ 27  17 293  23  84  19  10]
 [ 11  26  16 356  62   9  10]
 [ 20  30  25  21 364  13  15]
 [ 34  20  19  23 113 243  20]
 [ 22  15  13  25  62  12 335]]
